In [1]:
!pip install keras-tuner


     |████████████████████████████████| 61kB 2.9MB/s 
  Created wheel for keras-tuner: filename=keras_tuner-1.0.1-cp36-none-any.whl size=73200 sha256=6ac27df96935f083ee329233cc65e110612977699870df5bd66cacfcb30b9622
  Stored in directory: /root/.cache/pip/wheels/b9/cc/62/52716b70dd90f3db12519233c3a93a5360bc672da1a10ded43
  Created wheel for terminaltables: filename=terminaltables-3.1.0-cp36-none-any.whl size=15356 sha256=31bcdd6c3e4b1cf66a0f522d9df6040a1a495cd94f7f22778dc3dabe4144e714
  Stored in directory: /root/.cache/pip/wheels/30/6b/50/6c75775b681fb36cdfac7f19799888ef9d8813aff9e379663e
Successfully built keras-tuner terminaltables


In [2]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

In [3]:
print(tf.__version__)

2.3.0


In [4]:
fashion_mnist = keras.datasets.fashion_mnist

In [5]:
(train_images,train_labels),(test_images,test_labels) =  fashion_mnist.load_data()

4423680/4422102 [==============================] - 0s 0us/step


In [6]:
train_images = train_images/255.0
test_images = test_images/255.0

In [7]:
train_images[0].shape

(28, 28)

In [8]:
train_images = train_images.reshape(len(train_images),28,28,1)
test_images = test_images.reshape(len(test_images
                                      ),28,28,1)

In [9]:
def build_model(hp):
  model = keras.Sequential([
                            keras.layers.Conv2D(
                                filters=hp.Int('conv_1_filter',min_value=32,max_value=128,step=16),
                                kernel_size=hp.Choice('conv_1_kernel',values=[3,5]),
                                activation = 'relu',
                                input_shape = (28,28,1)
                                ),
                            keras.layers.Conv2D(
                                filters=hp.Int('conv_2_filter',min_value=32,max_value=64,step=16),
                                kernel_size=hp.Choice('conv_2_kernel',values=[3,5]),
                                activation = 'relu'
                            ),
                            keras.layers.Flatten(),
                            keras.layers.Dense(
                                units = hp.Int('dense_1_units',min_value=32,max_value=128,step=16),
                                activation='relu'
                            ),
                            keras.layers.Dense(10,activation='softmax')

  ])
  model.compile(optimizer=keras.optimizers.Adam(hp.Choice('learning_rate',values=[1e-2,1e-3])),
                loss='sparse_categorical_crossentropy',
                metrics=['accuracy']
                
                )
  return model

In [10]:
from kerastuner import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameters

In [11]:
tuner_search = RandomSearch(build_model,
                            objective='val_accuracy',max_trials=5,directory='output',project_name='Mnist Fashion')

In [12]:
tuner_search.search(train_images,train_labels,epochs=3,validation_split=0.1)

Epoch 1/3
1688/1688 [==============================] - 7s 4ms/step - loss: 0.4750 - accuracy: 0.8298 - val_loss: 0.3364 - val_accuracy: 0.8732
Epoch 2/3
1688/1688 [==============================] - 7s 4ms/step - loss: 0.3599 - accuracy: 0.8666 - val_loss: 0.3644 - val_accuracy: 0.8632
Epoch 3/3
1688/1688 [==============================] - 8s 5ms/step - loss: 0.3299 - accuracy: 0.8778 - val_loss: 0.3601 - val_accuracy: 0.8760


Epoch 1/3
1688/1688 [==============================] - 9s 5ms/step - loss: 0.4688 - accuracy: 0.8300 - val_loss: 0.4070 - val_accuracy: 0.8527
Epoch 2/3
1688/1688 [==============================] - 9s 5ms/step - loss: 0.3476 - accuracy: 0.8721 - val_loss: 0.3303 - val_accuracy: 0.8795
Epoch 3/3
1688/1688 [==============================] - 9s 5ms/step - loss: 0.3069 - accuracy: 0.8853 - val_loss: 0.3212 - val_accuracy: 0.8825


Epoch 1/3
1688/1688 [==============================] - 9s 5ms/step - loss: 0.5862 - accuracy: 0.7938 - val_loss: 0.4425 - val_accuracy: 0.8377
Epoch 2/3
1688/1688 [==============================] - 9s 5ms/step - loss: 0.4232 - accuracy: 0.8434 - val_loss: 0.4352 - val_accuracy: 0.8412
Epoch 3/3
1688/1688 [==============================] - 9s 5ms/step - loss: 0.3927 - accuracy: 0.8545 - val_loss: 0.4374 - val_accuracy: 0.8318


Epoch 1/3
1688/1688 [==============================] - 9s 5ms/step - loss: 0.4259 - accuracy: 0.8456 - val_loss: 0.3139 - val_accuracy: 0.8847
Epoch 2/3
1688/1688 [==============================] - 9s 5ms/step - loss: 0.2821 - accuracy: 0.8964 - val_loss: 0.2882 - val_accuracy: 0.8942
Epoch 3/3
1688/1688 [==============================] - 10s 6ms/step - loss: 0.2300 - accuracy: 0.9150 - val_loss: 0.2593 - val_accuracy: 0.9027


Epoch 1/3
1688/1688 [==============================] - 9s 5ms/step - loss: 0.4013 - accuracy: 0.8541 - val_loss: 0.3074 - val_accuracy: 0.8858
Epoch 2/3
1688/1688 [==============================] - 9s 5ms/step - loss: 0.2623 - accuracy: 0.9024 - val_loss: 0.2581 - val_accuracy: 0.9050
Epoch 3/3
1688/1688 [==============================] - 9s 5ms/step - loss: 0.2065 - accuracy: 0.9224 - val_loss: 0.2724 - val_accuracy: 0.9007


INFO:tensorflow:Oracle triggered exit


In [14]:
model = tuner_search.get_best_models(num_models=1)[0]

In [16]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 24, 24, 80)        2080      
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 20, 20, 48)        96048     
_________________________________________________________________
flatten (Flatten)            (None, 19200)             0         
_________________________________________________________________
dense (Dense)                (None, 128)               2457728   
_________________________________________________________________
dense_1 (Dense)              (None, 10)                1290      
Total params: 2,557,146
Trainable params: 2,557,146
Non-trainable params: 0
_________________________________________________________________


In [17]:
model.fit(train_images,train_labels,epochs=10,validation_split=0.1,initial_epoch=3)

Epoch 4/10
1688/1688 [==============================] - 9s 5ms/step - loss: 0.2073 - accuracy: 0.9221 - val_loss: 0.2522 - val_accuracy: 0.9115
Epoch 5/10
1688/1688 [==============================] - 9s 5ms/step - loss: 0.1607 - accuracy: 0.9392 - val_loss: 0.2902 - val_accuracy: 0.9055
Epoch 6/10
1688/1688 [==============================] - 9s 5ms/step - loss: 0.1216 - accuracy: 0.9541 - val_loss: 0.2778 - val_accuracy: 0.9107
Epoch 7/10
1688/1688 [==============================] - 9s 5ms/step - loss: 0.0920 - accuracy: 0.9664 - val_loss: 0.2974 - val_accuracy: 0.9093
Epoch 8/10
1688/1688 [==============================] - 10s 6ms/step - loss: 0.0694 - accuracy: 0.9742 - val_loss: 0.3356 - val_accuracy: 0.9053
Epoch 9/10
1688/1688 [==============================] - 9s 5ms/step - loss: 0.0545 - accuracy: 0.9798 - val_loss: 0.3906 - val_accuracy: 0.9115
Epoch 10/10
1688/1688 [==============================] - 9s 5ms/step - loss: 0.0436 - accuracy: 0.9846 - val_loss: 0.4463 - val_accurac